<a href="https://colab.research.google.com/github/JITHIN-ANTONY-JOSEPH/ERP_11358080/blob/main/9_Experiment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Input : Recipe Instructions
### Model : Word2Vec(Text) + Node2Vec(Graph) + Attention Based Embeddings Neural Networks(Model)

### Mounting to connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Installing required libraries

In [ ]:
!pip install node2vec

### Importing required libraries , loading datasets and pre-processing

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity  # Add this import

In [ ]:
flavorgraph_df = pd.read_csv('/content/drive/My Drive/ERP/Dataset/nodes_191120.csv')  # Adjust the path as needed , this is the path to my personal Google Drive

In [ ]:
import pandas as pd
import json
import re
import jellyfish
from gensim.models import Word2Vec
from concurrent.futures import ProcessPoolExecutor

# Load the main dataset
with open('/content/drive/My Drive/ERP/modified_Processed_Layer1.json', 'r') as file:# Adjust the path as needed , this is the path to my personal Google Drive
    recipe1m_data = [json.loads(line) for line in file]

recipe1m_df = pd.DataFrame(recipe1m_data)

# Load the substitution pairs
substitution_pairs_df = pd.read_csv('/content/drive/My Drive/ERP/Recipe1MSubs_full.csv')# Adjust the path as needed , this is the path to my personal Google Drive

# Merge the datasets based on recipe_id (substitution_pairs_df) and id (recipe1m_df)
merged_df = pd.merge(recipe1m_df, substitution_pairs_df, left_on='id', right_on='recipe_id')

# Example ingredient list for NER-like extraction (replace with your own comprehensive list or use NER model)
ingredient_list = set(flavorgraph_df[flavorgraph_df['node_type'] == 'ingredient']['name'].dropna().unique())

# Function to extract ingredients from instructions
def extract_ingredients_from_instructions(instructions, ingredient_list):
    extracted_ingredients = []
    for instruction in instructions:
        words = instruction.split()
        for word in words:
            if word in ingredient_list:
                extracted_ingredients.append(word)
    return extracted_ingredients

# Apply the extraction function
recipe1m_df['extracted_ingredients'] = recipe1m_df['processed_instructions'].apply(
    lambda instructions: extract_ingredients_from_instructions(instructions, ingredient_list) if isinstance(instructions, list) else []
)

# Prepare sentences for training
sentences = recipe1m_df['extracted_ingredients'].tolist()

# Add substitution contexts to sentences
for _, row in substitution_pairs_df.iterrows():
    ingredient1 = row['ingredient1']
    ingredient2 = row['ingredient2']
    sentences.append([ingredient1, ingredient2])

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=8)  # Increase 'workers' to utilize more CPU cores

### Generating graph embeddings

In [ ]:
import networkx as nx
from node2vec import Node2Vec

# Load the knowledge graph
flavor_graph = nx.read_graphml('/content/drive/My Drive/ERP/knowledge_graph.graphml') # Adjust the path as needed , this is the path to my personal Google Drive

# Filter the graph to include only ingredient nodes
ingredient_nodes = [n for n, attr in flavor_graph.nodes(data=True) if attr['node_type'] == 'ingredient']
flavor_graph = flavor_graph.subgraph(ingredient_nodes)

# Generate Node2Vec embeddings considering edge weights
node2vec = Node2Vec(flavor_graph, dimensions=100, walk_length=30, num_walks=200, workers=4, weight_key='weight')
graph_model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Generate graph embeddings for the ingredients
graph_embeddings = {str(node): graph_model.wv[str(node)] for node in flavor_graph.nodes()}

Computing transition probabilities:   0%|          | 0/6651 [00:00<?, ?it/s]

### Attention based combined embeddings and defining the combined model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import jellyfish

# Assuming text_embeddings contains Word2Vec embeddings and graph_embeddings contains Node2Vec embeddings
text_embeddings = model.wv  # Word2Vec model
graph_embeddings = {str(node): graph_model.wv[str(node)] for node in flavor_graph.nodes()}  # Node2Vec model

# Define a neural network with attention mechanisms to combine text and graph embeddings
class CombinedNNWithAttention(nn.Module):
    def __init__(self, text_dim, graph_dim, combined_dim):
        super(CombinedNNWithAttention, self).__init__()
        self.text_fc = nn.Linear(text_dim, combined_dim)
        self.graph_fc = nn.Linear(graph_dim, combined_dim)
        self.attention = nn.Parameter(torch.randn(2, 1))  # Adjust shape to (2, 1)

    def forward(self, text_embedding, graph_embedding):
        text_proj = self.text_fc(text_embedding)
        graph_proj = self.graph_fc(graph_embedding)

        combined = torch.stack([text_proj, graph_proj], dim=1)  # Shape: (batch_size, 2, combined_dim)
        attention_weights = F.softmax(torch.matmul(combined, self.attention), dim=1)  # Shape: (batch_size, 2, 1)

        weighted_sum = (combined * attention_weights).sum(dim=1)  # Shape: (batch_size, combined_dim)
        return weighted_sum

# Instantiate the attention-based model
combined_model = CombinedNNWithAttention(text_dim=100, graph_dim=100, combined_dim=128)

# Example of getting combined embeddings using the attention-based model
def get_combined_embedding_with_attention(ingredient, text_embeddings, graph_embeddings, model):
    text_embedding = torch.tensor(text_embeddings[ingredient] if ingredient in text_embeddings else np.zeros(100), dtype=torch.float32)
    graph_embedding = torch.tensor(graph_embeddings[ingredient] if ingredient in graph_embeddings else np.zeros(100), dtype=torch.float32)
    combined_embedding = model(text_embedding, graph_embedding)
    return combined_embedding

# Combine embeddings for training data
train_data = []
train_labels = []

for _, row in substitution_pairs_df.iterrows():
    ing1 = row['ingredient1']
    ing2 = row['ingredient2']
    combined_embedding1 = get_combined_embedding_with_attention(ing1, text_embeddings, graph_embeddings, combined_model)
    combined_embedding2 = get_combined_embedding_with_attention(ing2, text_embeddings, graph_embeddings, combined_model)

    train_data.append(combined_embedding1.detach().numpy())
    train_labels.append(combined_embedding2.detach().numpy())

train_data = torch.tensor(train_data, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.float32)

# Define the neural network for training
class CombinedNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CombinedNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.5)
        self.batch_norm = nn.BatchNorm1d(hidden_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.batch_norm(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Instantiate and train the improved model
improved_nn_model = CombinedNN(input_dim=128, hidden_dim=256, output_dim=128)
criterion = nn.MSELoss()
optimizer = optim.Adam(improved_nn_model.parameters(), lr=0.001)

# Training loop
for epoch in range(50):  # Adjust epochs as needed
    improved_nn_model.train()
    optimizer.zero_grad()
    outputs = improved_nn_model(train_data)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Evaluate on validation set
val_data = []
val_labels = []

for idx, row in enumerate(substitution_pairs_df.itertuples()):
    if idx >= 500:
        break
    ing1 = row.ingredient1
    combined_embedding = get_combined_embedding_with_attention(ing1, text_embeddings, graph_embeddings, combined_model)

    val_data.append(combined_embedding.detach().numpy())
    val_labels.append(row.ingredient2)

val_data = np.array(val_data)
val_data = torch.tensor(val_data, dtype=torch.float32)
val_predictions = improved_nn_model(val_data).detach().numpy()



<ipython-input-7-1b1669861525>:53: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  train_data = torch.tensor(train_data, dtype=torch.float32)


Epoch 1, Loss: 1.1614018678665161
Epoch 2, Loss: 1.0876461267471313
Epoch 3, Loss: 1.0271925926208496
Epoch 4, Loss: 0.9745786786079407
Epoch 5, Loss: 0.9298862814903259
Epoch 6, Loss: 0.8939566016197205
Epoch 7, Loss: 0.8621975183486938
Epoch 8, Loss: 0.8344807028770447
Epoch 9, Loss: 0.8104305267333984
Epoch 10, Loss: 0.7878848910331726
Epoch 11, Loss: 0.7688062191009521
Epoch 12, Loss: 0.7495964169502258
Epoch 13, Loss: 0.7323254942893982
Epoch 14, Loss: 0.7157021164894104
Epoch 15, Loss: 0.7008469700813293
Epoch 16, Loss: 0.6874053478240967
Epoch 17, Loss: 0.6741859912872314
Epoch 18, Loss: 0.6612924337387085
Epoch 19, Loss: 0.649154543876648
Epoch 20, Loss: 0.6395578980445862
Epoch 21, Loss: 0.6286355257034302
Epoch 22, Loss: 0.6188615560531616
Epoch 23, Loss: 0.6088233590126038
Epoch 24, Loss: 0.6010745763778687
Epoch 25, Loss: 0.5926738977432251
Epoch 26, Loss: 0.5845348238945007
Epoch 27, Loss: 0.5770642161369324
Epoch 28, Loss: 0.5698931813240051
Epoch 29, Loss: 0.563034296035

### Evaluation

In [ ]:
# Prepare validation data for the first 100 entries
val_data = []
val_labels = []

for idx, row in enumerate(substitution_pairs_df.itertuples()):
    if idx >= 500:
        break
    ing1 = row.ingredient1
    combined_embedding = get_combined_embedding_with_attention(ing1, text_embeddings, graph_embeddings, combined_model)

    val_data.append(combined_embedding.detach().numpy())
    val_labels.append(row.ingredient2)

val_data = np.array(val_data)
val_data = torch.tensor(val_data, dtype=torch.float32)

# Generate predictions
improved_nn_model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    val_predictions = improved_nn_model(val_data).detach().numpy()

# Function to find the top N most similar ingredients based on cosine similarity
def find_top_similar_ingredients(predicted_embedding, combined_embeddings, top_n=10):
    similarities = {}
    for ingredient, embedding in combined_embeddings.items():
        similarity = cosine_similarity(predicted_embedding.reshape(1, -1), embedding.reshape(1, -1))[0][0]
        similarities[ingredient] = similarity
    sorted_ingredients = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    return [ingredient for ingredient, similarity in sorted_ingredients[:top_n]]

# Function to calculate metrics with Jaro-Winkler similarity threshold
def calculate_metrics(predictions, ground_truths, combined_embeddings, top_n=10, threshold=0.8):
    mrr, hit_1, hit_3, hit_10 = 0.0, 0.0, 0.0, 0.0
    total = len(ground_truths)

    for pred, gt in zip(predictions, ground_truths):
        top_similar = find_top_similar_ingredients(pred, combined_embeddings, top_n=top_n)
        for rank, candidate in enumerate(top_similar, start=1):
            sim = jellyfish.jaro_winkler_similarity(gt, candidate)
            if sim >= threshold:
                mrr += 1.0 / rank
                if rank == 1:
                    hit_1 += 1.0
                if rank <= 3:
                    hit_3 += 1.0
                if rank <= 10:
                    hit_10 += 1.0
                break

    mrr /= total
    hit_1 /= total
    hit_3 /= total
    hit_10 /= total
    return mrr, hit_1, hit_3, hit_10

# Calculate metrics for the first 100 entries of the validation set
val_labels_str = val_labels  # Assuming labels are ingredient names
combined_embeddings = {ingredient: get_combined_embedding_with_attention(ingredient, text_embeddings, graph_embeddings, combined_model).detach().numpy() for ingredient in text_embeddings.index_to_key}
mrr, hit_1, hit_3, hit_10 = calculate_metrics(val_predictions, val_labels_str, combined_embeddings)

print(f"MRR: {mrr:.4f}, Hit@1: {hit_1:.4f}, Hit@3: {hit_3:.4f}, Hit@10: {hit_10:.4f}")

MRR: 0.1071, Hit@1: 0.0560, Hit@3: 0.1300, Hit@10: 0.2440
